In [1]:
from bs4 import BeautifulSoup
import datetime
import urllib.request
from selenium import webdriver

m = 1
while True :
    print('------------')
    print(m)
    url1 = 'https://www.coursera.org/search?query=data%20science&indices%5Bprod_all_products_term_optimization%5D%5BrefinementList%5D%5BallLanguages%5D=&indices%5Bprod_all_products_term_optimization%5D%5BrefinementList%5D%5BentityTypeDescription%5D=&indices%5Bprod_all_products_term_optimization%5D%5Bpage%5D='
    url2 = str(m)
    url3 = 'indices%5Bprod_all_products_term_optimization%5D%5Bconfigure%5D%5BclickAnalytics%5D=true&indices%5Bprod_all_products_term_optimization%5D%5Bconfigure%5D%5BhitsPerPage%5D=10&configure%5BclickAnalytics%5D=true'
    url = url1 + url2 +url3
    ourUrl=urllib.request.urlopen(url)
    soup = BeautifulSoup(ourUrl, 'html.parser')
    print(url)
    m += 1
    
    for course in soup.find_all('li',{'class':'ais-InfiniteHits-item'}):
        type1=course.find('div',{'class':'card-info vertical-box'})
        type2=type1.find('div',{'class':'productTypePill_vy0uva-o_O-gradientBg_98hw3a'})
        #print(type2)
        if not type2 == None:
            type=type2.contents[0]
            #print(type)
            if type=='COURSE':
                coursetag=course.find('a')
                course_url1=coursetag.get('href')
                print(course_url1)
                course_url2='https://www.coursera.org'+str(course_url1)
                course_url3=urllib.request.urlopen(course_url2)
                course_soup=BeautifulSoup(course_url3,'html.parser')
                html = course_url3.read().decode('utf-8')
                
                
                 # course hour*4
                course_time_total=0
                for course_time in course_soup.find_all('strong',{'class':'Strong_gjs17i-o_O-weightBold_uvlhiv-o_O-fontBody_56f0wi text-secondary'}):
                    #print('%%%%course_time%%%%')
                    course_t=course_time.getText().split()
                    if course_t[1]=='hour' or course_t[1]=='hours':
                        hour=course_t[0]
                        #print(hour)
                        course_time_total=course_time_total+int(hour)
                    else:
                        print('Time unit is not hour')
                        course_time_total='0'
                print('This course takes',course_time_total,'hours')
                
                #instructor1=course_soup.find('div',{'class':'Instructors'})
                #if not instructor1 == None:
                    #instructor2=instructor1.find('a',{'class':'link-no-style'})
                    #instructor=instructor2.getText()
                    #print(instructor)
                    
                # Instructor name
                instructor = []
                instructor_box = course_soup.find('div',{'class':'Instructors'})
                if instructor_box:
                    instructor_all = instructor_box.find_all('div',{'class':'Col_i9j08c-o_O-xsCol12_1m1ceo5-o_O-lgCol6_8hn8ps p-y-1 p-r-1'})
                    for i in instructor_all:
                        instructor_name = i.find('a',{'class':'link-no-style'}).getText()
                        instructor.append(instructor_name)    
                    print(instructor)
                else:
                    print('None')
                
                 #number of recent views
                infobox1 = course_soup.find('div',{'class':'Col_i9j08c-o_O-xsCol12_1m1ceo5'})
                recent_view = infobox1.find('div',{'class':'rc-ProductMetrics'})
                recent_view_number =recent_view.find('span').contents[0].getText()
                print(recent_view_number)

                
                #爬课程难易度，beginner等
                level1 = course.find('span', class_='difficulty')
                if level1 == None:
                    pass
                else:
                    level=course.find('span', class_='difficulty').text       
                print(level)
                
                #language
                lang = []
                language_box = course_soup.find_all('div',{'class':'Box_120drhm-o_O-displayflex_poyjc m-b-2 p-t-1s'})
                for i in language_box:
                    language_tag = i.find('h4',{'class':'H4_1k76nzj-o_O-weightBold_uvlhiv-o_O-bold_1byw3y2 m-b-0'})
                    if not language_tag == None:
                        language = language_tag.getText()
                        lang.append(language)
                if 'English' in lang:
                    print('English')
                else:
                    print('not English')
                
                #skills you will gain
                skill_box = course_soup.find('div',{'class':'Skills border-a p-x-2 p-t-1 p-b-2 m-y-2'})
                if not skill_box == None:
                    skill_tag = skill_box.find_all('span',{'class':'Pill_56iw91 m-r-1s m-b-1s'})
                    for s in skill_tag:
                        skill = s.find('span',{'class':'centerContent_dqfu5r'}).getText()
                        print(skill)
              
                
                #爬课程发布单位
                organization = course.find('span',class_='partner-name').text
                print(organization)
                
                #爬rating
                rating = course.find('span',class_='ratings-text').text
                print(rating)
                
                #爬评星数
                rating_number = course.find('span',class_='ratings-count').text
                print(rating_number)
                
                #爬评论数
                reviews_number = course_soup.find('div',class_='reviewsCount').text
                print(reviews_number)
                
                #爬enrollment数量                
                driver = webdriver.Chrome()
                driver.get(course_url2)
                enrollment_soup = BeautifulSoup(driver.page_source)
                enrollment_number = enrollment_soup.find('span',{'data-reactid':'177'}).text
                print(enrollment_number)
            
                #爬具体评论
                url_review1 = course_soup.find('a',class_='horizontal-box m-y-1').get('href')
                url_review2 = "https://www.coursera.org" + url_review1 +"?sort=recent&page="
                page = 1
                while True:
                    url_review = url_review2 +str(page)
                    page += 1
                    print(url_review)
                    review_url = urllib.request.urlopen(url_review)
                    review_soup = BeautifulSoup(review_url,'html.parser')
                    review_soup1 = review_soup.find_all('div', class_= 'rc-CML font-lg styled')
                    if review_soup1 == []:
                        break
                    for i in review_soup1:
                        review_content = i.text
                        print("@@@"+review_content)
                        
                    #for i in review_soup.find_all('div', class_= 'rc-CML font-lg styled'):
                        #review_content = i.text
                        #print("@@@"+review_content)
                    
                  
                    #爬最早时间
                    for j in review_soup.find_all('p', class_= 'dateOfReview p-x-1s m-b-0 text-secondary font-xs'):
                        review_time = j.text

                    
                print("$$$"+review_time)  #不用tab对齐
                
                #处理日期，转化英文日期到数字日期，计算课程总共开设了多少天
                timeformat = datetime.datetime.strptime(review_time,'%b %d, %Y')
                print(timeformat)
                today = datetime.datetime.strptime('2019-11-23 00:00:00', '%Y-%m-%d %H:%M:%S')
                course_duration = today - timeformat
                print(course_duration)
                

------------
1
https://www.coursera.org/search?query=data%20science&indices%5Bprod_all_products_term_optimization%5D%5BrefinementList%5D%5BallLanguages%5D=&indices%5Bprod_all_products_term_optimization%5D%5BrefinementList%5D%5BentityTypeDescription%5D=&indices%5Bprod_all_products_term_optimization%5D%5Bpage%5D=1indices%5Bprod_all_products_term_optimization%5D%5Bconfigure%5D%5BclickAnalytics%5D=true&indices%5Bprod_all_products_term_optimization%5D%5Bconfigure%5D%5BhitsPerPage%5D=10&configure%5BclickAnalytics%5D=true
/learn/what-is-datascience
This course takes 6 hours
['Alex Aklson', 'Polong Lin']
815,857
Beginner
English
IBM
4.7
(18,947)
3,081 reviews
135,984
https://www.coursera.org/learn/what-is-datascience/reviews?sort=recent&page=1
@@@Gives a very broad and meaningful vision about data science
@@@The most important element of this course is share the ideas and perceptions of people enrolled in data science world. It's a new but amazing new fied of work and study!
@@@It's very basic

KeyboardInterrupt: 